## Execution Query Plans Lab
> Download the dataset from [the official TLC Trip Record Data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

---

### This cell only shows how to document code
```python
# Load file
local_file = 'datasets/your-downloaded-from-TLC-taxis-file-here.parquet'

# Show data
spark.read.parquet(local_file).show()
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, year

### What is master(local N)?
The --master option specifies the master URL for a distributed cluster, or local to run locally with one thread, or local[N] to run locally with N threads.

<b>Source</b>: See Spark [docs here](spark.apache.org/docs/latest). See all [options here](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls)

In [ ]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

In [ ]:
# Read non-partitioned taxi data
local_files = 'datasets/parquet/'
df_taxis_non_partitioned_raw = spark.read.parquet(local_files)

In [ ]:
# Because we cleaned the data in the previous notebook, let's do the same:
df_taxis_non_partitioned_raw = df_taxis_non_partitioned_raw.where(year(col('tpep_pickup_datetime')) == '2023')

Note: Make sure you execute the following, from the previous notebook, to generate the partitioned data:

```python
    df_sink = df_clean_s1.withColumn("p_date",to_date(col('tpep_pickup_datetime')))
    df_sink.write.partitionBy("p_date").mode("append").parquet("datasets/yellow_taxis_daily/")
```

In [ ]:
# Read partitioned taxi data
local_path = 'datasets/yellow_taxis_daily/'
df_taxis_daily_raw = spark.read.parquet(local_path)

In [ ]:
# Show schema and find new partition column
df_taxis_daily_raw.printSchema()

In [ ]:
# Show new partition column
df_taxis_daily_raw.select('p_date').show(n=3)

In [ ]:
# Create same column p_date, so we can compare plans
df_taxis_nopartitioned_raw = df_taxis_non_partitioned_raw.withColumn("p_date",to_date(col('tpep_pickup_datetime')))

In [ ]:
# Register Non-partitioned DF as View
df_taxis_nopartitioned_raw.createOrReplaceTempView("tbl_taxis_nopartitioned_raw")

In [ ]:
# Register Daily DF as View
df_taxis_daily_raw.createOrReplaceTempView("tbl_taxis_daily_raw")

In [ ]:
# Query by partition Key; i.e. using '2023-02-14' as filter
q1a = spark.sql("select avg(trip_distance) from tbl_taxis_daily_raw where p_date='2023-02-14' and RatecodeID=2")

In [ ]:
# Show data
q1a.show()

In [ ]:
# Explain plan
q1a.explain(extended=True)

In [ ]:
# Query by partition Key; i.e. using '2023-02-14' as filter
q1b = spark.sql("select avg(trip_distance) from tbl_taxis_nopartitioned_raw where p_date='2023-02-14' and RatecodeID=2")

In [ ]:
# Explain plan
q1b.explain(extended=True)

In [ ]:
# Query by partition Key; i.e. using '2023-02-14' as filter
q2a = spark.sql("select p_date,count(1) from tbl_taxis_daily_raw where p_date in ('2023-02-14','2023-02-15','2023-02-16') group by p_date")


In [ ]:
# Query by partition Key; i.e. using '2023-02-14' as filter
q2b = spark.sql("select p_date,count(1) from tbl_taxis_nopartitioned_raw where p_date in ('2023-02-14','2023-02-15','2023-02-16') group by p_date")

In [ ]:
# Show plan
q2a.explain(extended=False)

In [ ]:
# Show plan
q2b.explain(extended=False)

---
### Compare performance

In [ ]:
# Show plan
q2a.explain(extended="formatted")

In [ ]:
# Show plan
q2b.explain(extended="formatted")

In [ ]:
%%timeit

# Query by partition Key; i.e. using '2023-02-14' as filter
spark.sql("select p_date,count(1) from tbl_taxis_daily_raw group by p_date order by to_date(p_date)").show(n=5)

In [ ]:
%%timeit

# Query by partition Key; i.e. using '2023-02-14' as filter
spark.sql("select p_date,count(1) from tbl_taxis_nopartitioned_raw group by p_date order by to_date(p_date)").show(n=5)

In [ ]:
# Stop the session
# spark.stop()